In [8]:
# TOOLS:
# 1. Wikipedia tool
# 2. Web Search tool
# 3. arxiv tool

In [9]:
# Importing required libraries and modules for the tools and Groq model
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_tavily import TavilySearch
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq
 
# Loading environment variables for API keys
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVALI_API_KEY = os.getenv("TAVALI_API_KEY")

In [10]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=5, doc_content_char_limit=3000)
arxiv_tool = ArxivQueryRun(
    arxiv_api_wrapper=arxiv_wrapper, 
    name="GroqArxivTool", 
    description="Useful for when you need to find research papers on arxiv.org to answer questions about scientific topics."
    )
print("Arxiv tool created with name: ", arxiv_tool.name)

arxiv_tool.invoke("Explain the concept of transformers in machine learning.")

Arxiv tool created with name:  GroqArxivTool


'Published: 2023-06-07\nTitle: Changing Data Sources in the Age of Machine Learning for Official Statistics\nAuthors: Cedric De Boom, Michael Reusens\nSummary: Data science has become increasingly essential for the production of official statistics, as it enables the automated collection, processing, and analysis of large amounts of data. With such data science practices in place, it enables more timely, more insightful and more flexible reporting. However, the quality and integrity of data-science-driven statistics rely on the accuracy and reliability of the data sources and the machine learning techniques that support them. In particular, changes in data sources are inevitable to occur and pose significant risks that are crucial to address in the context of machine learning for official statistics.\n  This paper gives an overview of the main risks, liabilities, and uncertainties associated with changing data sources in the context of machine learning for official statistics. We provi

In [11]:
wikipedia_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_char_limit=3000)
wikipedia_tool = WikipediaQueryRun(
    api_wrapper=wikipedia_wrapper, 
    name="GroqWikipediaTool", 
    description="Useful for when you need to look up general information on Wikipedia to answer questions about various topics."
    )

print("Wikipedia tool created with name: ", wikipedia_tool.name)

Wikipedia tool created with name:  GroqWikipediaTool


In [12]:


tavily_wrapper = TavilySearchAPIWrapper(tavily_api_key=TAVALI_API_KEY)
tavily_tool = TavilySearchResults(
    api_wrapper=tavily_wrapper, 
    max_results=5,                 
    name="GroqTavilySearchTool", 
    description="Useful for looking up recent information on the web."
)

print("Tavily Search tool created with name: ", tavily_tool.name)

Tavily Search tool created with name:  GroqTavilySearchTool


/var/folders/jx/2dy4zf5n1tv425bvn8023r5r0000gn/T/ipykernel_9765/1013946376.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(


In [13]:
# Combining all tools into a list
tools = [arxiv_tool, wikipedia_tool, tavily_tool]

# Initializing the Groq LLM
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0.01,
    max_tokens=5000,
    timeout=None,
    max_retries=2,
) 

AGENT = llm.bind_tools(tools)

In [14]:
from typing_extensions import TypedDict 
from langchain_core.messages import AnyMessage              # Human or AI message
from typing import Annotated                                # Labelling
from langgraph.graph.message import add_messages

from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition              # If my LLM is calling the tool then what path to take and when it is not calling what path the LLM has to take.

In [30]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]             # List of all AI and Human messages

# NODE DEFINATIONS
def tool_calling_llm(state: State):
    # AGENT: It is the LLM with tools bound to it.
    return {"messages": [AGENT.invoke(state["messages"])]}


builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition                                                 # Condition to check if the LLM is calling a tool or not. It has been predifened in langgraph.prebuilt. If there is not tool then LLM will go to END node.
)
builder.add_edge("tools", "tool_calling_llm")

CHATBOT = builder.compile()

# To visualize the graph
# display(Image(CHATBOT.get_graph().draw_mermaid_png()))

In [26]:
output = CHATBOT.invoke({"messages": "Can you give me the plot of hindi movie Ready"})

for o in output["messages"]:
    o.pretty_print()

================================ Human Message =================================

Can you give me the plot of hindi movie Ready
================================== Ai Message ==================================
Tool Calls:
  GroqWikipediaTool (twtmzmwm8)
 Call ID: twtmzmwm8
  Args:
    query: Ready (film) plot
================================= Tool Message =================================
Name: GroqWikipediaTool

Page: Ready or Not (2019 film)
Summary: Ready or Not is a 2019 American comedy horror film directed by Matt Bettinelli-Olpin and Tyler Gillett, and written by Guy Busick and R. Christopher Murphy. It stars Samara Weaving, Adam Brody, Mark O'Brien, Elyse Levesque, Nicky Guadagni, Henry Czerny and Andie MacDowell. It follows a young bride who is hunted by her spouse's wealthy family as part of a wedding night ritual to worship the Devil.
Preparations began in November 2017 when Bettinelli-Olpin and Gillett, members of the filmmaking collective Radio Silence, were hired as directo